# Welcome to the notebook

### Task 1 - Set up the project environment

In [ ]:
!pip install openai==1.7.2 python-dotenv

Importing modules

In [ ]:
import pandas as pd
import os, time
from openai import OpenAI
from dotenv import load_dotenv
from PIL import Image
from io import BytesIO

print("Modules are imported.")

Setting up the OpenAI API:

* Prepare a .env file to store the OpenAI API key.
* Uploading the .env file to our colab environment
* Load the API key and setup the API

In [ ]:
load_dotenv('apikey.env.txt')

APIKEY = os.getenv("APIKEY")
ORGID = os.getenv("ORGID")

Creating OpenAI Client

In [ ]:
client = OpenAI(
    api_key=APIKEY,
    organization=ORGID
)

print("OpenAI Client is ready.", client)

### Task 2 - Create the AI Assistant

Now let's load our dataset into a File Object

In [ ]:
data_file = client.files.create(
    file=open("product_sales_dataset.json", "rb"),
    purpose='assistants'
)

print("File uploaded:", data_file)

Let's define our Assistant object.

In [ ]:
name = "Data Analyst"

description = """
  You are a data analyst.
  You know how to analyze data and extract insights.
  You will be given a json file containing the data related to an online toy store.
  And you use your data analysis and coding skills to naswer the user's analytical questions about this data.
"""

assistant = client.beta.assistants.create(
    name=name,
    description=description,
    model="gpt-4-1106-preview",
    tools=[{"type": "code_interpreter"}],
    file_ids=[data_file.id]
)

print("Assistant created:", assistant)

### Task 3 - Communicate with the AI Assistant through Threads

> The **User** can communicate with the AI assistant through a **Thread** object. A thread can contain one or multiple **Message** objects. <br>
> We can then use a **Run** object to send our message to the AI assistant.




Let's define an empty thread. 🧵


In [ ]:
thread = client.beta.threads.create()

print("Thread created:", thread)

Now let's create a message and add it to thread.

In [ ]:
question = "Which product has the highest number of sold items?"

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = question
)

print("Message created:", message)

Creating a `Run` object to send our request to the AI Assistant.

In [ ]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)

print("Run created:", run)

### Task 4 - Retrieve results from the AI Assistant

Let's retrieve the run object to check the status of the run

In [ ]:
response_run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

print("Run status:", response_run.status)

Now let's retrieve the messages from the AI Assistant

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)

#print("Messages retrieved:", list(messages))

#print("Message content:", messages.data[0].content[0].text.value)

for message in list(messages):
  print(message.role + ": " + message.content[0].text.value)

### Task 5 - Create charts with AI Assistant

In [ ]:
def send_message(assistant_id, thread_id, message_content):
      """
      Sends a message to an AI assistant,
      waits for the response run to be completed,
      and then retrieves the list of messages in the thread.
      """

      # creating message
      message = client.beta.threads.messages.create(
        thread_id = thread_id,
        role = "user",
        content = message_content
      )

      # creating request run
      request_run = client.beta.threads.runs.create(
            thread_id = thread_id,
            assistant_id = assistant_id
      )

      # wait until the response_run is completed
      while True:
            # retrieve the run
            response_run = client.beta.threads.runs.retrieve(
                  thread_id = thread_id,
                  run_id = request_run.id
            )

            print("status > "+response_run.status)

            time.sleep(1)

            if(response_run.status=="completed"):
              time.sleep(1)
              break

      messages = client.beta.threads.messages.list(
            thread_id = thread_id
      )

      return list(messages)

In [ ]:
question = "In which day of week we have the highest number of sold items?"

messages = send_message(assistant.id, thread.id, question)

#print("Messages retrieved:", messages)

for message in messages:
  print(message.role + ": " + message.content[0].text.value)

Let's ask the assistant to create a histogram of the product price column.

In [ ]:
question = "Create a histogram of the product price column."

messages = send_message(assistant.id, thread.id, question)

answer = messages[0]
print(answer)

Let's take a look at the answer and extract the image file

In [ ]:
file_id = answer.content[0].image_file.file_id

print("File ID:", file_id)

Let's retrieve the file

In [ ]:
image_data = client.files.content(file_id).read()

image = Image.open(BytesIO(image_data))
image

Exercise: Create a line chart of the average number of sold items per month

In [ ]:
quesqtion = "Create a line chart of the average number of sold items per month."

messages = send_message(assistant.id,
                        thread.id,
                        question)

answer = messages[0]

file_id = answer.content[0].image_file.file_id

image_data = client.files.content(file_id).read()
image = Image.open(BytesIO(image_data))
image